#  Load libraries

In [1]:
import torch
import matplotlib.pylab as py
import numpy as np
from ipywidgets import interact
torch.manual_seed(123)

# Sparse matrix operator

$\mathrm{Init}$ is initialization,
$\Omega$ is the mask,
and $U$ is the update.  $U$ is the only thing that has a gradient

The operator is

$$
S = \mathrm{Init} + \Omega \odot U
$$

where $\odot$ is the Hadamard product.


In [2]:
class SparseMatrix(object):
    def __init__(self, U=None, Init=None, Omega=None, Init_noise=0.1):
        if U is None:
            self.U = torch.zeros((3, 3), requires_grad=True)
        else:
            self.U = U

        # Note, Init_noise == 0 is either a saddle or a maximum, so 
        # the gradient is 0 there and we can't optimize. We add a little
        # noise to get away from that point
        if Init is None:
            self.Init = torch.rand((3, 3), requires_grad=False)*Init_noise
        else:
            self.Init = U

        if Omega is None:
            self.Omega = torch.tensor([[1, 1, 1],
                                       [1, 1, 1],
                                       [1, 1, 1]], requires_grad=False)
        else:
            self.Omega = Omega
    
    def __call__(self, X):
        return (self.Init+self.Omega*self.U) @ X        


# Training data

We start with the 3-dimensional case.  So, 

$$\mathrm{Init} + \Omega \odot U \in \mathbb{R}^{3 \times 3}$$ 

so the data $X \in \mathbb{R}^{3 \times n}$ where $n$ is the number of training examples.

The equation we want to solve is

$$
(\mathrm{Init} + \Omega \odot U) X = Y
$$

In [3]:
X = torch.tensor([[1.],
                  [0.],
                  [0.]], requires_grad=False)

In [4]:
Y = torch.tensor([[0.],
                  [0.],
                  [2.]], requires_grad=False)

## An exact solution

We can compute the exact solution

$ (\mathrm{Init} + \Omega \odot U)X = Y $

Assume $\Omega = \mathbf{1}$ so $\Omega \odot U = U$.

$$ (\mathrm{Init} + U)X = Y $$
$$ \mathrm{Init}X + UX = Y $$
$$ UX = Y - \mathrm{Init}X $$
$$ U = (Y -\mathrm{Init}X)X^{-1} $$


Now, the equation above needs to be interpreted carfully.  

If $X \in \mathbb{R}^{3 \times 3}$ the inverse requires that $X$ is full rank.

If $X \in \mathbb{R}^{3 \times k}$ with $k<3$ then there are multiple possible values for $U$, given by appending arbitrary (though full rank) columns to $X$.

If $X \in \mathbb{R}^{3 \times k}$ with $k>3$ then the mapping is only minimal in a least-squares and $X^{-1}$ is really a pseudo-inverse.



In [5]:
with torch.no_grad():
    # Make pad out X to make it square
    X_padded = torch.hstack((X,torch.rand((3,2))))
    # Make a sparse matrix
    S = SparseMatrix(Init_noise=0.0)
    # This is one possible U_exact
    U_exact = (Y-S.Init @ X_padded) @ X_padded.inverse()
    # Make the exact solution
    S_exact = SparseMatrix(U=U_exact, Init_noise=0.0)
    print('This should be the 0 vector\n', S_exact(X) - Y )

This should be the 0 vector
 tensor([[0.],
        [0.],
        [0.]])


# Training the function

I am going to try to put the "training data" into the activation function. This needs more explanation, but I am not sure I understand it yet :-)

In [6]:
def activation(X, iteration):
    #X[1, :] = X[1, :]**3
    return X

In [7]:
def loss_function(Y, YHat, iteration):
    # This is the Forbenius norm
    return torch.linalg.matrix_norm(Y - YHat, ord='fro')

In [8]:
# We put this outside so that we can run the loop many times.
S = SparseMatrix()

In [9]:
optimizer = torch.optim.SGD([S.U], lr=0.01)
# The outer loop is the number of training epochs
epochs = 401
iterations = 2
for i in range(epochs):
    optimizer.zero_grad()
    interim_loss = 0
    # The inner loop is the number of iterations
    YHat = X.clone()
    for k in range(iterations):
        YHat = activation(S(YHat), k)
        interim_loss += loss_function(Y, YHat, k)

    final_loss = loss_function(Y, YHat, k)
    
    loss = 1*final_loss + 0*interim_loss

    if i%(epochs//10)==0:
        print(f'epoch {i} loss {loss}')

    if loss > 0.01:
        loss.backward()
        optimizer.step()
    else:
        print(f'epoch {i} final loss {loss}')
        break
        


epoch 0 loss 1.9845165014266968
epoch 40 loss 1.957511067390442
epoch 80 loss 1.8826569318771362
epoch 120 loss 1.6755435466766357
epoch 160 loss 1.1391788721084595
epoch 200 loss 0.15699271857738495
epoch 206 final loss 0.002248641336336732


In [10]:
X = torch.tensor([[1.],
                  [0.],
                  [0.]], requires_grad=False)
with torch.no_grad():
    for k in range(iterations):
        X = activation(S(X), k)
X

tensor([[-7.8893e-04],
        [ 2.1037e-03],
        [ 1.9999e+00]])

In [11]:
YHat

tensor([[-7.8893e-04],
        [ 2.1037e-03],
        [ 1.9999e+00]], grad_fn=<MmBackward0>)

# Non-linear

Now we take the next step.  We want to have a simple non-linear activation function.  The simplest
being something like

